**Assignment 2**

Gurjot Singh (B00811724)

Bhavneet Kaur (B00809769)

**Question 1**

*Part a*

Tokenize and perform part-of-speech tags

In [1]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
cats = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
news = fetch_20newsgroups(subset = 'train', categories = cats)

In [2]:
from nltk.tokenize import word_tokenize
import nltk
df = pd.DataFrame(news.data, columns = ['text'])
df['tokenized_data'] = df.apply(lambda row: nltk.word_tokenize(row["text"]), axis=1)
df['POSTags'] = nltk.pos_tag_sents(df['text'].apply(word_tokenize).tolist())


In [3]:
tokens = []
for i in news.data:
    temp = set(word_tokenize(i))
    tokens.append(temp)
flat_list = [item for sublist in tokens for item in sublist]

*Part b*

Extract collocation using Frequency with filter, PMI, T-test with filter,Chi-Sq test

In [4]:
bigrams = nltk.collocations.BigramAssocMeasures()
bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(flat_list)

In [5]:
bigrams_freq = bigramFinder.ngram_fd.items()
bigramFreqTable = pd.DataFrame(list(bigrams_freq), columns=['bigram','Frequency']).sort_values(by='Frequency', ascending=False)

In [6]:
from nltk.corpus import stopwords
en_stopwords = set(stopwords.words('english'))
#function to filter for ADJ/NN bigrams
def rightTypes(ngram):
    if '@' in ngram or '<' in ngram or '>' in ngram or 'Re' in ngram or 'Subject' in ngram:
        return False
    for word in ngram:
        if word in en_stopwords or word.isspace():
            return False
    acceptable_types = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    second_type = ('NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in acceptable_types and tags[1][1] in second_type:
        return True
    else:
        return False
#filter bigrams
filtered_bi = bigramFreqTable[bigramFreqTable.bigram.map(lambda x: rightTypes(x))]

In [7]:
bigrams_pmi = bigramFinder.score_ngrams(bigrams.pmi)   
bigramPmiTable = pd.DataFrame(list(bigrams_pmi), columns=['bigram','PMI']).sort_values(by='PMI', ascending=False)

In [9]:
bigrams_t = bigramFinder.score_ngrams(bigrams.student_t)
bigramTtable = pd.DataFrame(list(bigrams_t), columns=['bigram','t']).sort_values(by='t', ascending=False)

In [10]:
filteredT_bi = bigramTtable[bigramTtable.bigram.map(lambda x: rightTypes(x))]

In [11]:
bigrams_chi = bigramFinder.score_ngrams(bigrams.chi_sq)
bigramChiTable = pd.DataFrame(list(bigrams_chi), columns=['bigram','chi-sq']).sort_values(by='chi-sq', ascending=False)

In [12]:
chi_bi = bigramChiTable[:20].bigram.values
t_bi = filteredT_bi[:20].bigram.values
pmi_bi = bigramPmiTable[:20].bigram.values
freq_bi = filtered_bi[:20].bigram.values

Output of all the four methods

In [13]:
bigramsCompare = pd.DataFrame([freq_bi, pmi_bi, t_bi, chi_bi]).T
bigramsCompare.columns = ['Frequency With Filter', 'PMI', 'T-test With Filter', 'Chi-Sq Test']
bigramsCompare

,Frequency With Filter,PMI,T-test With Filter,Chi-Sq Test
0,"(writes, nothing)","('-g, SHOgraphics)","(writes, nothing)","('-g, SHOgraphics)"
1,"(Lines, life)","(conforming, whilst)","(Lines, life)","(complies, 841099)"
2,"(message, Organization)","(comp.sys, alogirhtm)","(message, Organization)","(compressibility, Wheeler)"
3,"(Institute, Lines)","(compacte, werkelijk)","(Institute, Lines)","(compressors, 155622)"
4,"(much, article)","(compatables, Paramus)","(much, article)","(concentrating, FREEDOM)"
5,"(Lines, word)","(compataible, dungeon.lonestar.org)","(Lines, word)","(condensing, 2*)"
6,"(access.digex.net, Organization)","(complies, 841099)","(access.digex.net, Organization)","(condor, Sen.)"
7,"(Distribution, Organization)","(compositions, Name___________________________...","(want, Christian)","(conductivity, fouled)"
8,"(want, Christian)","(compressibility, Wheeler)","(Nntp-Posting-Host, version)","(confirms, captivity)"
9,"(Nntp-Posting-Host, version)","(compressors, 155622)","(today, article)","(conforming, whilst)"


*Part c*

There is overlap between pairs of techniques, namely, 
1. Frequency with filer and T-test with filter
2. PMI and Chi-sq Test

Hence, using the union of these combination of techniques only makes sense as they have similar results.

**Question 2**

*Part a*

Cleaning the text

In [14]:
import re
import string
def clean_text(data): 
        temp = data.lower()
        number_temp = re.sub(r'\d+', ' ', temp)
        result = number_temp.translate(str.maketrans(" "," ", string.punctuation))
        return result

In [15]:
cleaned_text = []
for i in news.data:
    temp = []
    temp.append(clean_text(i))
    cleaned_text.append(temp)

In [16]:
df_cleaned = pd.DataFrame(cleaned_text, columns = ['data'])

In [17]:
df_cleaned['cleaned_data'] = df_cleaned['data'].apply(lambda x: ' '.join([word for word in x.split() if word not in (en_stopwords)])) 

In [18]:
df_cleaned['tokenized_data'] = df_cleaned.apply(lambda row: nltk.word_tokenize(row["cleaned_data"]), axis=1)

In [19]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
df_cleaned["stem_data"] = df_cleaned["tokenized_data"].apply(lambda x: [stemmer.stem(y) for y in x])
df_cleaned['final_data'] = df_cleaned['stem_data'].apply(' '.join)

*Part b*

Bag-of-words tf-idf weighted vector representation

In [20]:
X = df_cleaned['final_data']
y = news.target

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.3)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorize_data = vectorizer.fit_transform(X_train)
vectors_test = vectorizer.transform(X_test)

In [23]:
vectorize_data.shape

(1423, 21008)

*Part c*

The accuracy of SVM using linear scale is bit better than Multinomial NB because of the margin SVM uses. SVM uses the maximum margin approach, and hence has better boundaries for classification with higher margin. Thus, giving better accuracy.

In [24]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
clf = MultinomialNB(alpha=.01)
clf.fit(vectorize_data, y_train)
out = clf.predict(vectors_test)
print ("F1-score",metrics.f1_score(y_test, out, average='macro'))
print ("Accuracy",metrics.accuracy_score(y_test,out))

F1-score 0.9514900794431145
Accuracy 0.9574468085106383


In [25]:
from sklearn.metrics import confusion_matrix
cm_nb = confusion_matrix(y_test, out)
print ("Confusion Matrix for Multinomial NB\n",cm_nb)

Confusion Matrix for Multinomial NB
 [[134   0   0   6]
 [  0 169   2   0]
 [  0   1 178   0]
 [ 13   1   3 104]]


In [26]:
from sklearn import svm
clf_svm_k = svm.SVC(kernel = 'linear',gamma='scale')
clf_svm_k.fit(vectorize_data, y_train)
pred_k = clf_svm_k.predict(vectors_test)
print ("F1-score",metrics.f1_score(y_test, pred_k, average='macro'))
print ("Accuracy",metrics.accuracy_score(y_test,pred_k))

F1-score 0.9596309067191687
Accuracy 0.9639934533551555


In [27]:
cm_svm_k = confusion_matrix(y_test, pred_k)
print ("Confusion Matrix for SVM using linear scale\n",cm_svm_k)

Confusion Matrix for SVM using linear scale
 [[133   1   0   6]
 [  0 171   0   0]
 [  0   2 177   0]
 [  6   5   2 108]]


In [28]:
clf_svm = svm.SVC(gamma='scale')
clf_svm.fit(vectorize_data, y_train)
pred = clf_svm.predict(vectors_test)
print ("F1-score",metrics.f1_score(y_test, pred, average='macro'))
print ("Accuracy",metrics.accuracy_score(y_test,pred))

F1-score 0.9377630772379671
Accuracy 0.9459901800327333


In [29]:
cm_svm = confusion_matrix(y_test, pred)
print ("Confusion Matrix for SVM\n",cm_svm)

Confusion Matrix for SVM
 [[133   1   0   6]
 [  0 171   0   0]
 [  0   2 177   0]
 [ 13   8   3  97]]


When using non-linear kernel, the accuracy is decreased a bit because of the difference in the hyperplane created when diving the plane into different decision boundaries.

*Part d*

The accuracy obtained is almost similar as compared to the complete text even though the vocabulary formed is comparably a lot less than the previous part.

In [30]:
df['POSTags'] = df['POSTags'].apply(lambda x: [word for (word,pos) in x if (pos == 'NN' or pos=='NNP' or pos=='NNPS' or pos=='NNS')])
df['noun_data'] = df['POSTags'].apply(' '.join)

In [31]:
noun_clean = []
for i in df['noun_data']:
    noun_clean.append(clean_text(i))

In [32]:
df['noun_clean'] = noun_clean

In [33]:
df['cleaned_data'] = df['noun_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (en_stopwords)])) 

In [34]:
df['tokenized_data'] = df.apply(lambda row: nltk.word_tokenize(row["cleaned_data"]), axis=1)

In [35]:
df["stem_data"] = df["tokenized_data"].apply(lambda x: [stemmer.stem(y) for y in x])
df['final_data'] = df['stem_data'].apply(' '.join)

In [36]:
X = df['final_data']
y = news.target

In [37]:
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.3)

In [38]:
vectorizer = TfidfVectorizer()
vectorize_data = vectorizer.fit_transform(X_train)

In [39]:
vectorize_data.shape

(1423, 16583)

In [40]:
vectors_test = vectorizer.transform(X_test)

In [41]:
clf = MultinomialNB(alpha=.01)
clf.fit(vectorize_data, y_train)
out = clf.predict(vectors_test)
metrics.f1_score(y_test, out, average='macro')

0.9512606243810668

In [42]:
cm_nb = confusion_matrix(y_test, out)
print (cm_nb)

[[137   1   0   6]
 [  0 178   2   2]
 [  0   2 182   0]
 [ 10   1   1  89]]


In [43]:
clf_svm = svm.SVC(gamma='scale')
clf_svm.fit(vectorize_data, y_train)
pred = clf_svm.predict(vectors_test)
metrics.f1_score(y_test, pred, average='macro')

0.9544921416209171

In [44]:
cm_svm = confusion_matrix(y_test, pred)
print (cm_svm)

[[137   2   2   3]
 [  0 182   0   0]
 [  0   6 178   0]
 [  6   6   0  89]]


In [45]:
clf_svm_k = svm.SVC(kernel = 'linear', gamma='scale')
clf_svm_k.fit(vectorize_data, y_train)
pred_k = clf_svm_k.predict(vectors_test)
metrics.f1_score(y_test, pred_k, average='macro')

0.9658238579853592

In [46]:
cm_svm_k = confusion_matrix(y_test, pred_k)
print (cm_svm_k)

[[139   0   2   3]
 [  0 181   1   0]
 [  0   5 179   0]
 [  5   2   1  93]]


**References:**

1. https://www.nltk.org/api/nltk.tokenize.html#nltk.tokenize.api.TokenizerI
2. https://www.programcreek.com/python/example/104689/sklearn.datasets.fetch_20newsgroups
3. https://medium.com/@nicharuch/collocations-identifying-phrases-that-act-like-individual-words-in-nlp-f58a93a2f84a
4. https://medium.com/@datamonsters/text-preprocessing-in-python-steps-tools-and-examples-bf025f872908
5. https://scikit-learn.org/stable/modules/svm.html
6. https://stackoverflow.com/questions/33587667/extracting-all-nouns-from-a-text-file-using-nltk
7. https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction
8. https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html
9. https://scikit-learn.org/0.19/auto_examples/model_selection/grid_search_text_feature_extraction.html#sphx-glr-auto-examples-model-selection-grid-search-text-feature-extraction-py
10. https://scikit-learn.org/0.19/auto_examples/text/document_classification_20newsgroups.html#sphx-glr-auto-examples-text-document-classification-20newsgroups-py
11. https://stackoverflow.com/questions/41674573/how-to-apply-pos-tag-sents-to-pandas-dataframe-efficiently/41674780
